In [ ]:
# First reset the pygkyl library
import sys
!rm -rf ~/personal_gkyl_scripts/pygkyl/pygkyl.egg-info
!rm -rf ~/personal_gkyl_scripts/pygkyl/build
!{sys.executable} -m pip install ~/personal_gkyl_scripts/pygkyl > ~/personal_gkyl_scripts/pygkyl/install.log

import numpy as np
import matplotlib.pyplot as plt
import os

# Configure plotting
plt.rcParams["figure.figsize"] = (6,4)

# Custom libraries and routines
import pygkyl

home_dir = os.path.expanduser("~")
repo_dir = home_dir+'/personal_gkyl_scripts/'
# simdir = repo_dir+'sim_data_dir_example/par_adv_test/'
# fileprefix = 'rt_gk_tcv_nt_iwl_3x2v_p1'
# simdir = 'sim_data_dir_example/3x2v_example/gk_tcv_posD_iwl_3x2v_electron_heating/'
# fileprefix = 'gk_tcv_posD_iwl_3x2v_D02'
simdir = '../../sim_data_dir_example/3x2v_example/gk_tcv_nt_high_res_geom/'
fileprefix = 'rt_gk_tcv_nt_iwl_3x2v_p1'
# simdir = 'sim_data_dir_example/2x2v_example/tcv_NTtq/wk/'
# fileprefix = 'gk_tcv_negD_trueq_iwl_2x2v'
simulation = pygkyl.simulation_configs.import_config('tcv_nt', simdir, fileprefix, dimensionality='3x2v')
# simulation = pygkyl.simulation_configs.import_config('d3d_nt', simdir, fileprefix)
# simulation.geom_param.x_LCFS = 0.08

simulation.normalization.set('t','mus') # time in micro-seconds
simulation.normalization.set('x','minor radius') # radial coordinate normalized by the minor radius (rho=r/a)
simulation.normalization.set('y','Larmor radius') # binormal in term of reference sound Larmor radius
simulation.normalization.set('z','pi') # parallel angle devided by pi
simulation.normalization.set('fluid velocities','thermal velocity') # fluid velocity moments are normalized by the thermal velocity
simulation.normalization.set('temperatures','eV') # temperatures in electron Volt
simulation.normalization.set('pressures','Pa') # pressures in Pascal
simulation.normalization.set('energies','MJ') # energies in mega Joules
simulation.normalization.set('gradients','major radius') # gradients are normalized by the major radius

sim_frames = simulation.available_frames['ion_BiMaxwellianMoments'] # you can check the available frames for each data type like ion_M0, ion_BiMaxwellian, etc.)
print("%g time frames available (%g to %g)"%(len(sim_frames),sim_frames[0],sim_frames[-1]))

### Field line plot
Here we plot the mesh connected along the field lines.

In [ ]:
import pyvista as pv
import numpy as np
import postgkyl as pg

simName = simulation.data_param.fileprefix
data = pg.GData(simName+"-nodes.gkyl")
vals = data.get_values()

plotter = pv.Plotter()

for ix0 in [0,-2]:
    for iy0 in [0,-2]:
        ix1 = ix0 + 1
        iy1 = iy0 + 1
        X = vals[ix0:ix1,iy0:iy1,:,0]
        Y = vals[ix0:ix1,iy0:iy1,:,1]
        Z = vals[ix0:ix1,iy0:iy1,:,2]
        # Now stack the grid points in Fortran (column-major) order
        points = np.stack((X, Y, Z), axis=-1).reshape(-1, 3, order='C')
        # points = np.stack((X, Y, Z), axis=-1).reshape(-1, 3)
        grid = pv.StructuredGrid()
        grid.points = points
        (nx, ny, nz) = X.shape
        grid.dimensions = (nx, ny, nz)
        
        # Visualize
        point_cloud = pv.PolyData(points)
        plotter.add_mesh(point_cloud, point_size=5.0, render_points_as_spheres=True)
        plotter.add_mesh(grid, color='black', style='wireframe', line_width=1.0)

plotter.show()

### Perpendiular planes
Here we plot the mesh connected along the perpendicular planes.

In [ ]:
import pyvista as pv
import numpy as np
import postgkyl as pg

simName = simulation.data_param.fileprefix
data = pg.GData(simName+"-nodes.gkyl")
vals = data.get_values()
Nx, Ny, Nz, Ndim = vals.shape
plotter = pv.Plotter()
print("Grid shape: %g x %g x %g"%(Nx, Ny, Nz))
iz0 = 0
dz = 1
ix0 = Nx//2
ix1 = Nx
dx = 1
iy0 = Ny//2
iy1 = iy0 + 3
dy = 1
for iz0 in range(0, Nz, dz):
# for iz0 in range(Nz//2, Nz//2 + 16):
    iz1 = iz0 + 1
    X = vals[ix0:ix1:dx,iy0:iy1:dy,iz0:iz1:dz,0]
    Y = vals[ix0:ix1:dx,iy0:iy1:dy,iz0:iz1:dz,1] 
    Z = vals[ix0:ix1:dx,iy0:iy1:dy,iz0:iz1:dz,2]


    # # Now stack the grid points in Fortran (column-major) order
    points = np.stack((X, Y, Z), axis=-1).reshape(-1, 3, order='F')

    grid = pv.StructuredGrid()
    grid.points = points
    (nx, ny, nz) = X.shape
    grid.dimensions = (nx, ny, nz)

    # Visualize
    point_cloud = pv.PolyData(points)
    plotter.add_mesh(point_cloud, point_size=5.0, render_points_as_spheres=True)
    plotter.add_mesh(grid, color='black', style='wireframe', line_width=1.0)

plotter.show()

### Point cloud
Here we plot only the nodes of the mesh as a point cloud. No connections are shown.

In [ ]:
import pyvista as pv
import numpy as np
import postgkyl as pg
import matplotlib.pyplot as plt
# cmap = plt.cm.plasma  # Choose your colormap (e.g. viridis, plasma, inferno, etc.)
cmap = plt.cm.get_cmap('tab10')  # 10 discrete colors

simName = simulation.data_param.fileprefix
data = pg.GData(simName+"-nodes.gkyl")
vals = data.get_values()
Nx, Ny, Nz, Ndim = vals.shape
plotter = pv.Plotter()

iz0 = 0
dz = 1
ix0 = 0# Nx//2
ix1 = Nx
dx = 1
iy0 = 0# Ny//2
iy1 = Ny# iy0 + 1
dy = 1
for iz0 in range(0, Nz-1, 1):
# for iz0 in range(Nz//2, Nz//2 + 16):
    iz1 = iz0 + 1
    X = vals[ix0:ix1:dx,iy0:iy1:dy,iz0:iz1:dz,0]
    Y = vals[ix0:ix1:dx,iy0:iy1:dy,iz0:iz1:dz,1] 
    Z = vals[ix0:ix1:dx,iy0:iy1:dy,iz0:iz1:dz,2]
    
    # t = iz0 / (Nz - 1 - 1)  # Normalize index to [0, 1]
    # color = cmap(t)  # Get RGBA color
    color = cmap(iz0 % cmap.N)  # Get discrete color from colormap

    # # Now stack the grid points in Fortran (column-major) order
    points = np.stack((X, Y, Z), axis=-1).reshape(-1, 3, order='F')

    grid = pv.StructuredGrid()
    grid.points = points
    (nx, ny, nz) = X.shape
    grid.dimensions = (nx, ny, nz)

    # Visualize
    point_cloud = pv.PolyData(points)
    plotter.add_mesh(point_cloud, color=color[:3], point_size=5.0)
    # plotter.add_mesh(grid, color='black', style='wireframe', line_width=1.0)

plotter.show()